### 외국어 학습에 도움이 될만한 게임 만들기

1. 학습할 품사를 고른다.
2. 주어진 예시 문장들을 SUMMARY 한다.
2. summary한 문장들의 마지막 문장에서 해당 품사에 해당하는 단어를 하나 가린다.
3. 해당 단어가 무엇인지 맞춰봅시다~

In [53]:
# flair 가 없는 경우 꼭 설치해주세요 
#!pip install flair

In [54]:
example_text = [
    'Die Familie macht OpenAI und Unternehmenschef Sam Altman verantwortlich. Der Vorwurf lautet, die damals eingesetzte Version des Sprachmodells ChatGPT-4o sei „trotz klarer Sicherheitsprobleme übereilt auf den Markt gebracht“ worden. Nach Angaben der Familie tauschte der Jugendliche zeitweise bis zu 650 Nachrichten pro Tag mit dem Chatbot aus. Der Anwalt der Familie, Jay Edelson, erklärte, der Tod des Jugendlichen sei „unausweichlich“ gewesen. OpenAI habe die eigenen Warnungen von Sicherheitsteams ignoriert. Auch der frühere Forschungsleiter Ilya Sutskever habe aus diesem Grund das Unternehmen verlassen. Die Anwälte wollen vor Gericht belegen, dass interne Bedenken zugunsten eines schnellen Marktstarts übergangen worden seien.',
    'Ein großer Astronaut aus Pappe weist in Berlin Mitte den Weg: Hier entlang bitte zum BM Future. So nennt die CSU-Leitung ihr Ministerium für Forschung, Technologie und Raumfahrt. Und tatsächlich fanden am vergangenen Wochenende viele Interessierte darunter auffallend viele Kinder den Weg in das Ministerium. Einen solchen Besucherandrang hat das eher unscheinbare Haus bisher selten gesehen. Seit dem Regierungswechsel ist das Ministerium auch für die Raumfahrt zuständig. Für den Tag der offenen Tür der Bundesregierung hatte Ministerin Dorothee Bär deshalb den Astronauten Alexander Gerst, auch bekannt als Astro-Alex, eingeladen. Er erzählte den staunenden kleinen Besuchern vom Leben auf der Weltraumstation. Zum Beispiel wie es ist, wenn man in der Schwerelosigkeit aufs Klo muss.',
    'Nach dem schweren russischen Luftangriff mit mehr als 20 Toten in Kyjiw hat der ukrainische Präsident Wolodymyr Selenskyj Moskau jeglichen Friedenswillen abgesprochen. »Er tötet Kinder, um nicht darüber sprechen zu müssen, wann und wie Frieden kommen wird, sagte Selenskyj am Donnerstagabend in seiner abendlichen Videobotschaft in Kyjiw. Nach dem nächtlichen Luftangriff, einem der schwersten in dreieinhalb Jahren Krieg, wurden in Kyjiw bis zum Donnerstagabend 21 Tote gezählt, darunter offenbar auch Kinder. Laut Behörden gab es rund 50 Verletzte. Der militärische Verwaltungschef der Hauptstadt, Tymur Tkatschenko sagte, dass in den Trümmern eines Wohnhauses vermutlich noch mehr Menschen verschüttet seien. Auch die EU-Vertretung in Kyjiw und das britische Kulturinstitut wurden beschädigt. Auf X schrieb Selenskyj später: Es war einer der größten Angriffe Russlands.'
    ]

In [55]:
from flair.models import SequenceTagger
from flair.data import Sentence
from transformers import pipeline
from nltk.tokenize import sent_tokenize
import time
import random

In [56]:
# 필요한 model을 다운로드하고 끌어오는 함수들

# 토크나이저 모델을 로드하는 함수
def make_tokenizer_model():
    tagger = SequenceTagger.load("flair/upos-multi")
    return tagger


# 요약 모델을 로드하는 함수
def make_summary_model():
    return pipeline('summarization', model='deutsche-telekom/mt5-small-sum-de-en-v1')


# text 요약하는 함수
def summarize_text(summary_model, text):
    # 입력받은 문장들을 요약
    summary = summary_model.__call__(text , return_text=True)
    # 요약한 문장들만 추출
    summary = summary[0]['summary_text']
    return summary
    

# 품사와 인덱스 사전 만드는 함수
def make_pos_word_dict(tagger,last_sentence):
    pos_word_dict = {}
    sentence = Sentence(last_sentence)
    tagger.predict(sentence)
    for token in sentence:
        token_str =f"{token.get_label()}"
        tokens = token_str.split()
        pos = tokens[3]
        word = tokens[1].replace('"','')
        if pos not in pos_word_dict:
            pos_word_dict[pos] = [word]
        else:
            pos_word_dict[pos].append(word)
    return pos_word_dict


# 품사 입력하면 그것에 해당하는 인덱스 랜덤으로 반환. 즉, <mask>할 인덱스와 단어 랜덤으로 반환
def select_random_word(pos_idx_dict,pos):
    # pos에 해당하는 값이 하나만 있으면 그것만 반환
    if len(pos_idx_dict[pos])==1:
        random_word = pos_idx_dict[pos][0]

    # 2개 이상이면 랜덤으로 반환
    else:
        random_word = random.choice(pos_idx_dict[pos])
    return random_word
    

# summary 문장을 받아 문제와 정답을 만들어 반환하는 함수 
def make_problem_solving(summary,mask_word):
    # 요약한 문장들을 문장 단위로 분리
    summary = sent_tokenize(summary)
    # summary에서 마지막 문장만 추출
    last_sentence = summary[-1]
    summary = summary[:-1]
    # 답이 될 단어를 🤔로 대체
    problem_sentence = last_sentence.replace(mask_word, '🤔')
    # <mask>를 추가한 문제 항목을 문자열로 합치기
    return summary, problem_sentence

##### 게임을 해봅시당~

In [57]:
tagger = make_tokenizer_model()
summary_model = make_summary_model()

2025-08-29 14:50:38,985 SequenceTagger predicts: Dictionary with 17 tags: NOUN, PUNCT, ADP, VERB, ADJ, DET, PROPN, ADV, PRON, AUX, CCONJ, NUM, SCONJ, PART, X, SYM, INTJ


c:\Users\Playdata\anaconda3\envs\nlp_env\Lib\site-packages\transformers\convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cpu


In [58]:
print('🎉단어 학습 게임에 오신 것을 환영합니다🎉')
print(f'오늘은 {len(example_text)}회 만큼 학습을 해보겠습니다.')

print("NOUN, VERB, DET 중에서 어떤 것을 학습해볼까요?")
while True:
    input_pos = input("위 목록에서 학습할 단어를 정확히 대문자로 입력하세요.")
    if input_pos not in ['NOUN','VERB','DET']:
        print('다시 정확히 입력하세요')
        continue
    else:
        print(f'좋습니다! 오늘은 "{input_pos}"를 학습해보겠습니다 :)')
        break

correct_cnt=0
incorrect_cnt=0
incorrect_word = []

for text in example_text:
    print('먼저 텍스트를 보여드리겠습니다.')
    print('📖요약하기 전 텍스트📖')
    print(text)
    print()

    print('✒️요약을 시작합니다!✒️')
    summary=summarize_text(summary_model, text)
    time.sleep(1)
    print('🎉요약이 완료되었습니다!🎉')
    print()
    
    last_sentence = sent_tokenize(summary)[-1]
    pos_word_dict = make_pos_word_dict(tagger,last_sentence)
    answer = select_random_word(pos_word_dict, input_pos)
    summary, problem_sentence = make_problem_solving(summary,answer)

    print('📖요약한 텍스트📖')
    print(summary)
    print()

    print('다음은 위 텍스트에 곧바로 이어질 문장입니다.')
    print(problem_sentence)
    print('🤔에 들어갈 알맞은 단어를 입력해봅시다!')
    guess = input('🤔에 들어갈 알맞은 단어를 입력하시오.')

    if guess == answer:
        print('😎정답입니다!')
        correct_cnt+=1
        print('😎다음 문제로 넘어가봅시다!')
        print()
        
    else:
        print(f'🥲틀렸습니다. 정답은 {answer}였습니다!')
        incorrect_cnt+=1
        incorrect_word.append(answer)


print(f'🤓정답률 : {correct_cnt / len(example_text):.2f}% 오늘 학습한 것 중 맞은 것의 개수 : {correct_cnt}, 틀린 것의 개수 : {incorrect_cnt}')

if len(incorrect_word)!=0:
    print(f'복습할 단어를 보여드립니다! {incorrect_word}')
else:
    print('🎉다 맞으셔서 복습할 단어가 없습니다!! 수고하셨습니다🩵')

🎉단어 학습 게임에 오신 것을 환영합니다🎉
오늘은 3회 만큼 학습을 해보겠습니다.
NOUN, VERB, DET 중에서 어떤 것을 학습해볼까요?
좋습니다! 오늘은 "NOUN"를 학습해보겠습니다 :)
먼저 텍스트를 보여드리겠습니다.
📖요약하기 전 텍스트📖
Die Familie macht OpenAI und Unternehmenschef Sam Altman verantwortlich. Der Vorwurf lautet, die damals eingesetzte Version des Sprachmodells ChatGPT-4o sei „trotz klarer Sicherheitsprobleme übereilt auf den Markt gebracht“ worden. Nach Angaben der Familie tauschte der Jugendliche zeitweise bis zu 650 Nachrichten pro Tag mit dem Chatbot aus. Der Anwalt der Familie, Jay Edelson, erklärte, der Tod des Jugendlichen sei „unausweichlich“ gewesen. OpenAI habe die eigenen Warnungen von Sicherheitsteams ignoriert. Auch der frühere Forschungsleiter Ilya Sutskever habe aus diesem Grund das Unternehmen verlassen. Die Anwälte wollen vor Gericht belegen, dass interne Bedenken zugunsten eines schnellen Marktstarts übergangen worden seien.

✒️요약을 시작합니다!✒️
🎉요약이 완료되었습니다!🎉

📖요약한 텍스트📖
['Die Familie macht OpenAI und Unternehmenschef Sam Altman verantwortlich.']

다음은 